# What is the doggest dog?

This is a notebook that can be used to perform experiments connected to our paper.

The aim of our method is to determine prototypes and anti-prototypes for a given basic-level category. Our methods aim to examine deep learning models from the perspective of the Prototype Theory.

Method from the paper: Strategy 1 (based on within-category similarity)

In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd

We use the following function to find the hyponyms for a given basic-level category - hypernym:

In [2]:
def find_hyponyms(hypernym_name):
    """
    Function ca be used to obtain a list of indexes of the ImageNet classes 
    belonging to a given basic-level category (a hypernym).
    ------
    Pramaters: 
    
    hypernym_name: a desired hypernym (e.g. "domestic_cat")
    """
    from nltk.corpus import wordnet as wn
    import nltk
    from tensorflow.keras.applications.imagenet_utils import decode_predictions
    from tensorflow.keras.utils import to_categorical
    import numpy as np
    
    hyponyms = wn.synsets(hypernym_name)[0]
    hyponyms = set([i for i in hyponyms.closure(lambda s:s.hyponyms())])
    offsets = []
    imagenet_classes = decode_predictions(to_categorical(np.expand_dims(np.array(range(1000)), axis=-1), num_classes=1000), top=1)

    for c in imagenet_classes:
        offsets.append(int(c[0][0].split('n')[1]))
    
    ids = []
    for idx, o in enumerate(offsets):
        isadoggo = wn.synset_from_pos_and_offset('n', int(o))
        if isadoggo in hyponyms:
            ids.append(idx)
    return np.array(ids)

In [3]:
def return_prototype_antiprototype(model, hypernym_name, keras=True, levit=False):
    """
    Function can be used to find prototypes and anti-prototypes for a given basic-level category 
    for a given deep learning model. 
    Parameters: 
    model: keras/torch model trained on ImageNet (with a standard order of classes that can be decoded with 
    tensorflow.keras.applications.imagenet_utils.decode_predictions
    hypernym_name: string with a name of a basic-level category, e.g. "domestic_cat", "dog" etc.
    high_level_category: in strategy 2, besides the hyponyms of our desired category, we also use
    a contrasting category. A contrasting category is a complement of a set of some higher-level categories. 
    keras: flag whether we want to use a keras model (set True if yes - it is a default). 
    Set false in the case of using a torch model.
    levit: set with keras=False in the case of using one of the LeViTs models.
    """
    from tensorflow.keras.applications.imagenet_utils import decode_predictions
    from sklearn.metrics.pairwise import cosine_similarity
    from tensorflow.keras.utils import to_categorical

    
    hyponym_classes = find_hyponyms(hypernym_name=hypernym_name)    
    print(f"Members of the basic-level category (for hypernym = {hypernym_name}):")
    print(len(hyponym_classes))
    
    # Extraction of weights (we consider models from Keras Application (https://keras.io/api/applications/)
    # and HuggingFace (https://huggingface.co/models) listed in the provided file Model_List.pdf
    if keras:
        hyponym_weights = np.mean(model.layers[-1].get_weights()[0][:, hyponym_classes], axis=1)
        hyponyms = np.moveaxis(model.layers[-1].get_weights()[0][:, hyponym_classes], -1, 0)
        hypernym_template = np.expand_dims(hyponym_weights, axis=0)
    else:
        if not levit:
            hyponym_weights = np.mean(model.classifier.weight.detach().numpy()[hyponym_classes, :], axis=0)
            hyponyms = model.classifier.weight.detach().numpy()[hyponym_classes, :]
            hypernym_template = np.expand_dims(hyponym_weights, axis=0)
        else:
            # models from the LeViT family have slightly different model structure than the other models 
            # from HuggingFace
            hyponym_weights = np.mean(model.classifier_distill.linear.weight.detach().numpy()[hyponym_classes, :], axis=0)
            hyponyms = model.classifier_distill.linear.weight.detach().numpy()[hyponym_classes, :]
            hypernym_template = np.expand_dims(hyponym_weights, axis=0)
            
    # Calculating the value of our similarity to find prototypes and anti-types

    squeezed_cs = np.squeeze(cosine_similarity(hyponyms, hypernym_template))
    
    print("prototype")
    most_similar = decode_predictions(np.expand_dims(to_categorical(hyponym_classes[np.argmax(squeezed_cs)], num_classes=1000), axis=0), top=1)[0][0]
    print(most_similar)
    print("anti-prototype")
    least_similar = decode_predictions(np.expand_dims(to_categorical(hyponym_classes[np.argmin(squeezed_cs)], num_classes=1000), axis=0), top=1)[0][0]
    print(least_similar)
    print()
    print("=========================================")
    print()
    return most_similar[1], least_similar[1]

Below one needs to set a WordNet node name - a desired basic-level category.

**In our experiments, we use the following nodes:**

Natural categories:

* *domestic_cat*
* *dog*
* *bird*
* *fish*
* *mammal*

Artificial category: *musical_instrument*

In [4]:
hypernym_name = "domestic_cat"

Example for keras:

In [5]:
# generating results for different CNNs (keras)

# ---------------------------------------------------------------------
model = tf.keras.applications.InceptionV3(
    include_top=True,
    weights="imagenet",
    pooling='avg',
    classes=1000,
    classifier_activation="softmax"
)

print(model.name)
most_similar, least_similar = return_prototype_antiprototype(model=model,  
                                                        hypernym_name=hypernym_name, 
                                                        keras=True)

inception_v3
Members of the basic-level category (for hypernym = domestic_cat):
5
prototype
('n02123045', 'tabby', 1.0)
anti-prototype
('n02123597', 'Siamese_cat', 1.0)




Example for torch models:

In [7]:
# generating results for different transformers (pytorch) - standard + example LeViT

from transformers import AutoModelForImageClassification

name = "microsoft/swinv2-tiny-patch4-window8-256"
model = AutoModelForImageClassification.from_pretrained(name)

print(name)
most_similar, least_similar = return_prototype_antiprototype(model=model,  
                                                        hypernym_name=hypernym_name, 
                                                        keras=False)

# ---------------------------------------------------------------

name = 'facebook/levit-384'
model = AutoModelForImageClassification.from_pretrained(name)

print(model.__class__.__name__)
most_similar, least_similar = return_prototype_antiprototype(model=model,  
                                                        hypernym_name=hypernym_name, 
                                                        keras=False, levit=True)

microsoft/swinv2-tiny-patch4-window8-256
Members of the basic-level category (for hypernym = domestic_cat):
5
prototype
('n02123045', 'tabby', 1.0)
anti-prototype
('n02123597', 'Siamese_cat', 1.0)


LevitForImageClassificationWithTeacher
Members of the basic-level category (for hypernym = domestic_cat):
5
prototype
('n02123045', 'tabby', 1.0)
anti-prototype
('n02123597', 'Siamese_cat', 1.0)


